#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [ ]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [ ]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
12864,-3.0,77,14.0,69.0,overcast,992.44,1,Winter,3.0,inland,Snowy
5569,25.0,70,14.0,48.0,overcast,1009.35,4,Winter,5.5,inland,Cloudy
2451,10.0,85,17.0,59.0,overcast,1010.03,1,Summer,2.0,mountain,Rainy
2340,-4.0,78,12.0,75.0,partly cloudy,984.50,0,Winter,3.5,mountain,Snowy
12777,30.0,76,14.5,25.0,partly cloudy,1015.22,2,Spring,8.5,mountain,Cloudy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [ ]:
# START CODE HERE

object_columns = data.select_dtypes(include=['object']).columns.tolist()
non_object_columns = data.select_dtypes(include=['int64','float64']).columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [ ]:

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import pandas as pd

# Identify object and non-object columns
object_columns = data.select_dtypes(include=['object']).columns.tolist()
non_object_columns = data.select_dtypes(exclude=['object']).columns.tolist()

# Define transformers for numerical and categorical columns
numerical_transformer = ('num', StandardScaler(), non_object_columns)
categorical_transformer = ('cat', OrdinalEncoder(), object_columns)

# Create ColumnTransformer
column_transformer = ColumnTransformer([
    numerical_transformer,
    categorical_transformer
])

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# Combine transformed data into a DataFrame
all_columns = non_object_columns + object_columns
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)



**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Failed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [ ]:
X = data_scaled_labeled.drop('Weather Type', axis=1)
y = data['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [ ]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,-0.179894,0.014320,-0.337587,0.300609,-0.520104,-0.878846,2.0,1.0,1.0,1.0
12278,1.085516,-0.926578,-1.061339,0.150333,1.035729,0.900845,3.0,3.0,1.0,3.0
2859,-0.122375,0.063841,0.603291,-0.137585,-0.520104,0.307615,2.0,1.0,0.0,0.0
5495,0.625367,-0.381848,-0.409962,0.126676,0.257813,0.900845,3.0,1.0,1.0,3.0
8469,0.050181,-1.817954,-0.699463,0.376957,1.035729,-0.137308,3.0,3.0,2.0,3.0


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [ ]:
# START CODE HERE

from sklearn.tree import DecisionTreeClassifier

model=DecisionTreeClassifier()


model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 100.0


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [ ]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,0.222737,-0.480889,-0.482337,0.553578,1.813646,0.307615,0.0,3.0,0.0,3.0
2331,-0.122375,-0.282806,0.675666,0.292544,-0.260799,0.010999,3.0,0.0,1.0,0.0
217,0.050181,0.063841,0.386166,0.066189,-0.779410,-0.433923,3.0,3.0,1.0,1.0
1760,-1.617860,1.301864,0.675666,-0.575778,-1.038715,-1.175461,2.0,3.0,2.0,2.0
3963,0.280255,1.301864,-0.120461,0.186087,-0.260799,-0.137308,2.0,1.0,1.0,1.0


In [ ]:
y = model.predict(input)
y

array(['Sunny', 'Cloudy', 'Rainy', 'Snowy', 'Rainy'], dtype=object)

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
